# Agent-to-Agent (A2A) Protocol

This notebook introduces the **Agent-to-Agent (A2A) Protocol**, a standardized communication protocol developed by Google for enabling seamless interaction between AI agents across different systems and platforms.

## What is the A2A Protocol?

The A2A protocol is an open standard that defines how AI agents can discover, communicate, and collaborate with each other, regardless of their underlying implementation or hosting infrastructure. Think of it as HTTP for AI agents.

### Key Features

- **Standardized Communication**: JSON-RPC 2.0 based protocol for agent-to-agent messaging
- **Agent Discovery**: Well-known endpoint (`/.well-known/agent.json`) for agent capabilities
- **Provider Agnostic**: Works with any AI provider (OpenAI, Azure OpenAI, Claude, local models, etc.)
- **Interoperability**: Enables agents from different vendors to work together
- **Protocol Compliance**: Includes validation and inspection tools

### Architecture Overview

```
┌─────────────────┐
│  Client Agent   │  (Your application with SK agents)
│  with A2A       │
│  Integration    │
└────────┬────────┘
         │ A2A Protocol (JSON-RPC 2.0)
         │
    ┌────┴────┬─────────┬──────────┐
    │         │         │          │
┌───▼───┐ ┌──▼───┐ ┌───▼────┐ ┌──▼─────┐
│Invoice│ │Policy│ │Logistics│ │Custom  │
│Agent  │ │Agent │ │Agent    │ │Agent   │
│Server │ │Server│ │Server   │ │Server  │
└───────┘ └──────┘ └─────────┘ └────────┘
  (A2A)    (A2A)     (A2A)       (A2A)
```

### Why Use A2A?

1. **Microservices for AI**: Decompose complex AI systems into specialized agents
2. **Reusability**: Share agents across different applications and teams
3. **Scalability**: Scale individual agents independently
4. **Security**: Isolate sensitive operations in dedicated agents
5. **Multi-Vendor**: Mix agents from different providers

> **Warning**: The A2A protocol is still under development and evolving rapidly. These examples use [SharpA2A.Core](https://www.nuget.org/packages/SharpA2A.Core) which tracks the protocol specification.

## References

- [A2A Protocol Specification](https://google.github.io/A2A/)
- [SharpA2A.Core NuGet Package](https://www.nuget.org/packages/SharpA2A.Core)
- [Semantic Kernel A2A Samples](https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/Demos/A2AClientServer)

## Setup and Configuration

In [ ]:
// Load configuration settings
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

Console.WriteLine($"Configuration loaded:");
Console.WriteLine($"- Model: {model}");
Console.WriteLine($"- Endpoint: {azureEndpoint}");
Console.WriteLine($"- API Key configured: {!string.IsNullOrEmpty(apiKey)}");

## Install Required Packages

In [ ]:
// Install Semantic Kernel and A2A packages
#r "nuget: Microsoft.SemanticKernel, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"
#r "nuget: SharpA2A.Core, 0.1.0"

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;  // Alias to avoid conflicts
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text.Json;

Console.WriteLine("Packages loaded successfully.");

## Understanding A2A Protocol Messages

The A2A protocol uses JSON-RPC 2.0 for message exchange. Let's examine the key message types:

### 1. Agent Discovery (Agent Card)

Agents expose their capabilities via a well-known endpoint:

```http
GET /.well-known/agent.json
```

Response:
```json
{
  "name": "Invoice Agent",
  "description": "Handles invoice-related queries",
  "version": "1.0",
  "capabilities": ["invoice_lookup", "invoice_validation"],
  "endpoints": {
    "message": "/"
  }
}
```

### 2. Sending Messages

```json
{
  "jsonrpc": "2.0",
  "id": "1",
  "method": "message/send",
  "params": {
    "id": "conv_123",
    "message": {
      "role": "user",
      "messageId": "msg_1",
      "parts": [{
        "kind": "text",
        "text": "Show me invoice for Contoso"
      }]
    }
  }
}
```

### 3. Receiving Responses

```json
{
  "jsonrpc": "2.0",
  "id": "1",
  "result": {
    "message": {
      "role": "assistant",
      "messageId": "msg_2",
      "parts": [{
        "kind": "text",
        "text": "Invoice ID: INV-001, Amount: $1500.00"
      }]
    }
  }
}
```

## Scenario: Multi-Agent Customer Service System

Let's build a realistic scenario where a customer service system uses multiple specialized agents via A2A protocol:

- **Invoice Agent**: Handles invoice lookup and validation
- **Policy Agent**: Provides company policies and procedures
- **Logistics Agent**: Tracks shipments and deliveries

### Conceptual Implementation

While we can't run full A2A servers in this notebook, we'll demonstrate the concepts using Semantic Kernel agents and show how they would integrate with A2A protocol.

## Creating Specialized Agents

First, let's create the specialized agents that would typically be exposed via A2A protocol:

In [ ]:
// Create the kernel for our agents
var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
Kernel kernel = builder.Build();

// Invoice Agent - Specializes in invoice-related queries
string invoiceAgentInstructions = @"
You are an Invoice Agent specializing in invoice lookup and validation.
You have access to invoice data and can:
- Look up invoices by ID or customer name
- Validate invoice details
- Provide invoice summaries

Sample invoice data:
- Invoice ID: INV-TICKET-XYZ987
- Customer: Contoso
- Date: June 18, 2025
- Items: 150 T-Shirts @ $10.00, 200 Hats @ $15.00, 300 Glasses @ $5.00
- Total: $7,500.00

Always be precise and reference specific invoice numbers.
";

ChatCompletionAgent invoiceAgent = new()
{
    Name = "InvoiceAgent",
    Instructions = invoiceAgentInstructions,
    Kernel = kernel
};

// Policy Agent - Provides company policies
string policyAgentInstructions = @"
You are a Policy Agent specializing in company policies and customer communications.

For short shipment disputes, always reply with exactly this policy:

Policy: Short Shipment Dispute Handling Policy V2.1

Summary: For short shipments reported by customers, first verify internal shipment records
(SAP) and physical logistics scan data (BigQuery). If discrepancy is confirmed and logistics data
shows fewer items packed than invoiced, issue a credit for the missing items. Document the
resolution in SAP CRM and notify the customer via email within 2 business days, referencing the
original invoice and the credit memo number. Use the 'Formal Credit Notification' email template.
";

ChatCompletionAgent policyAgent = new()
{
    Name = "PolicyAgent",
    Instructions = policyAgentInstructions,
    Kernel = kernel
};

// Logistics Agent - Tracks shipments
string logisticsAgentInstructions = @"
You are a Logistics Agent specializing in shipment tracking and delivery information.

For shipment queries, always reply with shipment details in this format:

Shipment number: SHPMT-SAP-001
Item: TSHIRT-RED-L
Quantity: 100
Status: Delivered
Delivery Date: June 20, 2025
";

ChatCompletionAgent logisticsAgent = new()
{
    Name = "LogisticsAgent",
    Instructions = logisticsAgentInstructions,
    Kernel = kernel
};

Console.WriteLine("Specialized agents created:");
Console.WriteLine($"- {invoiceAgent.Name}");
Console.WriteLine($"- {policyAgent.Name}");
Console.WriteLine($"- {logisticsAgent.Name}");

## Creating a Coordinator Agent with A2A Integration

In a real A2A implementation, this coordinator would:
1. Discover remote agents via `/.well-known/agent.json`
2. Send JSON-RPC messages to remote agents
3. Aggregate responses from multiple agents

For this demonstration, we'll simulate the A2A protocol behavior:

In [ ]:
// Coordinator Agent - Orchestrates calls to specialized agents
string coordinatorInstructions = @"
You are a Customer Service Coordinator that helps resolve customer issues by consulting
with specialized agents.

You have access to three specialized agents via A2A protocol:
- InvoiceAgent: For invoice-related queries
- PolicyAgent: For company policies and procedures  
- LogisticsAgent: For shipment and delivery information

When a customer has an issue:
1. Analyze what information you need
2. Determine which agents to consult
3. Request information from the appropriate agents
4. Synthesize the responses into a coherent solution

Always explain your reasoning and cite which agent provided which information.
";

ChatCompletionAgent coordinatorAgent = new()
{
    Name = "CoordinatorAgent",
    Instructions = coordinatorInstructions,
    Kernel = kernel
};

Console.WriteLine($"Coordinator agent created: {coordinatorAgent.Name}");

## Simulating A2A Protocol Interactions

Let's simulate how the coordinator would interact with specialized agents using A2A protocol:

In [ ]:
// Helper function to simulate A2A protocol message exchange
async Task<string> CallAgentViaA2A(Agent agent, string query, string conversationId)
{
    Console.WriteLine($"\n[A2A Protocol] Calling {agent.Name}");
    Console.WriteLine($"[A2A Protocol] Conversation ID: {conversationId}");
    Console.WriteLine($"[A2A Protocol] Query: {query}");
    
    // In real A2A, this would be a JSON-RPC 2.0 message:
    // POST http://agent-url/
    // { "jsonrpc": "2.0", "method": "message/send", "params": {...} }
    
    var history = new ChatHistory();
    history.AddUserMessage(query);
    
    var response = "";
    await foreach (var message in agent.InvokeAsync(history))
    {
        response += message.Content;
    }
    
    Console.WriteLine($"[A2A Protocol] Response from {agent.Name}:");
    Console.WriteLine(response);
    
    return response;
}

Console.WriteLine("A2A protocol simulation helper created.");

## Running a Multi-Agent Scenario

Let's handle a customer dispute that requires coordination across multiple agents:

In [ ]:
// Customer issue
string customerIssue = @"
Customer is disputing transaction TICKET-XYZ987 as they claim they received 
fewer t-shirts than ordered. They want to know what to do.
";

Console.WriteLine("Customer Issue:");
Console.WriteLine("===============");
Console.WriteLine(customerIssue);
Console.WriteLine();

// Generate a conversation ID (in A2A this would be managed by the protocol)
string conversationId = $"conv_{Guid.NewGuid():N}";

// Step 1: Consult Invoice Agent for transaction details
string invoiceQuery = "Look up transaction TICKET-XYZ987 and provide invoice details, especially the quantity of t-shirts ordered.";
string invoiceResponse = await CallAgentViaA2A(invoiceAgent, invoiceQuery, conversationId);

// Step 2: Consult Logistics Agent for shipment details
string logisticsQuery = "Check shipment details for transaction TICKET-XYZ987, specifically the quantity of t-shirts shipped.";
string logisticsResponse = await CallAgentViaA2A(logisticsAgent, logisticsQuery, conversationId);

// Step 3: Consult Policy Agent for dispute handling procedures
string policyQuery = "What is our policy for handling short shipment disputes?";
string policyResponse = await CallAgentViaA2A(policyAgent, policyQuery, conversationId);

// Step 4: Coordinator synthesizes the information
Console.WriteLine("\n" + "=".PadRight(80, '='));
Console.WriteLine("COORDINATOR SYNTHESIS");
Console.WriteLine("=".PadRight(80, '='));

string coordinatorQuery = $@"
Based on the following information from our specialized agents, provide a complete
resolution for the customer:

Customer Issue: {customerIssue}

Invoice Agent Response:
{invoiceResponse}

Logistics Agent Response:
{logisticsResponse}

Policy Agent Response:
{policyResponse}

Provide a clear action plan for resolving this dispute.
";

var coordinatorHistory = new ChatHistory();
coordinatorHistory.AddUserMessage(coordinatorQuery);

await foreach (var message in coordinatorAgent.InvokeAsync(coordinatorHistory))
{
    Console.Write(message.Content);
}

## A2A Protocol Benefits Demonstrated

In this example, we've demonstrated the key benefits of the A2A protocol:

### 1. **Separation of Concerns**
- Each agent has a specific responsibility
- Agents can be developed and maintained independently
- Easy to add new specialized agents

### 2. **Scalability**
- Individual agents can be scaled based on load
- Agents can be distributed across multiple servers
- No single point of failure

### 3. **Reusability**
- Same agents can serve multiple applications
- Agents can be shared across teams
- Reduces duplication of AI implementations

### 4. **Vendor Independence**
- Each agent can use different AI models
- Mix cloud and local models
- Switch providers without changing integration code

### 5. **Security & Compliance**
- Sensitive operations isolated in dedicated agents
- Fine-grained access control
- Easier to audit and comply with regulations

## Real A2A Implementation

To implement actual A2A servers, you would:

### 1. Create A2A Server Projects

```csharp
// Using SharpA2A.Core and ASP.NET Core
var builder = WebApplication.CreateBuilder(args);

// Configure A2A agent
builder.Services.AddA2AAgent(
    name: "InvoiceAgent",
    description: "Handles invoice queries",
    agentHandler: /* Your Semantic Kernel agent */
);

var app = builder.Build();
app.MapA2AEndpoints();
app.Run();
```

### 2. Expose Agent Card

```csharp
// Automatically exposed at /.well-known/agent.json
app.MapGet("/.well-known/agent.json", () => new AgentCard
{
    Name = "InvoiceAgent",
    Description = "Handles invoice queries",
    Version = "1.0.0",
    Capabilities = new[] { "invoice_lookup", "invoice_validation" }
});
```

### 3. Implement A2A Client

```csharp
// Connect to remote A2A agents
var a2aClient = new A2AClient();
await a2aClient.DiscoverAgent("http://localhost:5000");

// Send message via A2A protocol
var response = await a2aClient.SendMessage(new A2AMessage
{
    ConversationId = "conv_123",
    Message = "Show me invoice TICKET-XYZ987"
});
```

### 4. Test with A2A Inspector

Use the [A2A Inspector](https://github.com/a2aproject/a2a-inspector) to validate your implementation:

```bash
docker run -p 8080:8080 a2aproject/a2a-inspector
```

Then navigate to http://localhost:8080 and enter your agent URL.

## Key Takeaways

1. **A2A enables microservices architecture for AI agents**
   - Decompose monolithic AI systems
   - Scale and maintain components independently

2. **Standardization enables interoperability**
   - Different teams can share agents
   - Mix vendors and technologies
   - Easier integration and testing

3. **Protocol-based communication is flexible**
   - HTTP-based (works with existing infrastructure)
   - JSON-RPC 2.0 (widely supported)
   - Language and framework agnostic

4. **Semantic Kernel integrates naturally**
   - Use SK agents as A2A servers
   - Leverage SK's plugin system
   - Combine with other SK features

## Next Steps

- Explore the [A2A specification](https://google.github.io/A2A/)
- Try the [Semantic Kernel A2A samples](https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/Demos/A2AClientServer)
- Build your own A2A server with SharpA2A.Core
- Test with the A2A Inspector tool
- See Assignment 4 for hands-on practice